In [1]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

In [2]:
# Étape 1 : Preprocessing et feature engineering de la base de donnée 2022 pour la rendre exploitable.

# Import des modules et fichiers.
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler

df_2022_full=pd.read_csv('../data/world-hapiness-report-2022.csv',sep=";")

# Suppression de toutes les lignes non relatives à 2022.
df_2022=df_2022_full[(df_2022_full['year']==2022)]

# Ajout des continents.
#df_continents=pd.read_csv('../data/pays&continents.csv',sep=';')
#df_2022=df_2022.merge(right=df_continents,on='Country name',how='outer')
#df_2022.insert(1,'Regional indicator',df_2022.pop('Regional indicator'))

# Suppression des valeurs manquantes.
df_2022=df_2022.dropna()

# Suppression de la colonne des années.
df_2022=df_2022.drop(['year'],axis=1)

# Séparation des variables explicatives et de la variable nom de pays de la variable cible.
y_test2022=df_2022.drop(['Life Ladder', 'Country name'],axis=1)
X_test2022=df_2022['Life Ladder']

#On encode les continents de y_test2022
#y_test2022=pd.get_dummies(y_test2022,columns=['Regional indicator'])

# Normalisation des données.
scaler=MinMaxScaler()
cols=['Log GDP per capita','Social support','Healthy life expectancy at birth','Freedom to make life choices',
      'Generosity','Perceptions of corruption','Positive affect','Negative affect']
y_test2022[cols]=scaler.fit_transform(y_test2022[cols])

# Suppression de la colonne "Regional indicator_Central and Eastern Europe" pour correspondre à l'encodage des données
# fait par le OneHotEncoding.
#y_test2022=y_test2022.drop(['Regional indicator_Central and Eastern Europe'],axis=1)

# X_test2022 et y_test2022 sont alors respectivements les jeux d'entraînement et de test.

In [3]:
# Étape 2 : Récupération des jeux d'entraînement et de test générés dans les notebooks précédents (données de 2006 
# à 2021).

# Import des données.
y_train=pd.read_csv('../data/X_train.csv')
y_test=pd.read_csv('../data/X_test.csv')
X_train=pd.read_csv('../data/y_train.csv')
X_test=pd.read_csv('../data/y_test.csv')

# Concaténation.
X_train2021=pd.concat([X_train,X_test],axis=0)
y_train2021=pd.concat([y_train,y_test],axis=0)

# Transofrmation du jeu y en tableau python.
#y_train2021=y_train2021['Life Ladder'].values

y_train2021=y_train2021.drop(['Regional indicator_Commonwealth of Independent States'],axis=1)
y_train2021=y_train2021.drop(['Regional indicator_East Asia'],axis=1)
y_train2021=y_train2021.drop(['Regional indicator_Latin America and Caribbean'],axis=1)
y_train2021=y_train2021.drop(['Regional indicator_Middle East and North Africa'],axis=1)
y_train2021=y_train2021.drop(['Regional indicator_North America and ANZ'],axis=1)
y_train2021=y_train2021.drop(['Regional indicator_South Asia'],axis=1)
y_train2021=y_train2021.drop(['Regional indicator_Southeast Asia'],axis=1)
y_train2021=y_train2021.drop(['Regional indicator_Sub-Saharan Africa'],axis=1)
y_train2021=y_train2021.drop(['Regional indicator_Western Europe'],axis=1)

# Normalisation des données.
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
cols=['Log GDP per capita','Social support','Healthy life expectancy at birth','Freedom to make life choices',
      'Generosity', 'Perceptions of corruption','Positive affect','Negative affect']
y_train2021[cols]=scaler.fit_transform(y_train2021[cols])

In [5]:
# Étape 3: Faire tourner le modele de Random forest pour la regression et le tester.

from sklearn.ensemble import RandomForestRegressor
rf_model = RandomForestRegressor(max_depth=30,max_features='sqrt',min_samples_leaf=1,min_samples_split=2,n_estimators=300)

X_test2022 = X_test2022.to_frame()

rf_model.fit(X_train2021,y_train2021)

print("Score sur jeu d'entrainement 2021 :",rf_model.score(X_train2021,y_train2021))
print("Score sur jeu de test 2022 :",rf_model.score(X_test2022,y_test2022))

Score sur jeu d'entrainement 2021 : 0.7808656855761213
Score sur jeu de test 2022 : 0.20599232592794753


In [8]:
from sklearn.linear_model import Ridge

ridge_reg = Ridge(alpha=1.0)
ridge_reg.fit(X_train2021, y_train2021)

ridge_predictions = ridge_reg.predict(X_test2022)

print("Score sur jeu d'entrainement 2021 :",ridge_reg.score(X_train2021,y_train2021))
print("Score sur jeu de test 2022 :",ridge_reg.score(X_test2022,y_test2022))

Score sur jeu d'entrainement 2021 : 0.31685178879601317
Score sur jeu de test 2022 : 0.23811159699124473


In [15]:
from sklearn.linear_model import ElasticNet

elastic_net = ElasticNet(alpha=1.0, l1_ratio=0.5)
elastic_net.fit(X_train2021, y_train2021)

en_predictions = elastic_net.predict(X_test2022)

print("Score sur jeu d'entrainement 2021 :",elastic_net.score(X_train2021,y_train2021))
print("Score sur jeu de test 2022 :",elastic_net.score(X_test2022,y_test2022))

Score sur jeu d'entrainement 2021 : -1.0547118733938987e-15
Score sur jeu de test 2022 : -0.10586792642938936
